# Langchain Permit Tools

This package provides two Langchain tools for JWT validation and permission checking using Permit.io:

* `LangchainJWTValidationTool`: Validates JWT tokens against a JWKS endpoint

* `LangchainPermissionsCheckTool`: Checks user permissions using Permit.io

## Installation

```bash
pip install langchain-permit
```

## Environment Variables

Set up the following environment variables:

```bash
PERMIT_API_KEY=your_permit_api_key
JWKS_URL=your_jwks_endpoint_url
PERMIT_PDP_URL=your_permit_pdp_url  # Usually http://localhost:7766 for local development or your real deployment
```

## Prerequisites

Make sure your PDP (Policy Decision Point) is running at PERMIT_PDP_URL.
See [Permit docs](https://docs.permit.io/concepts/pdp/overview/) for details on policy setup and how to launch the PDP container.


## JWT Validation Tool
The JWT Validation tool verifies JWT tokens against a JWKS (JSON Web Key Set) endpoint.

### Basic Usage

```python
from langchain_permit.tools import LangchainJWTValidationTool

# Initialize the tool
jwt_validator = LangchainJWTValidationTool(
    jwks_url="http://your-jwks-endpoint/.well-known/jwks.json"
)

# Validate a token
async def validate_token():
    claims = await jwt_validator._arun(
        "eyJhbGciOiJSUzI1NiI..."  # Your JWT token
    )
    print("Validated Claims:", claims)
```

## Configuration Options
You can initialize the tool with either:

* A JWKS URL
* Direct JWKS JSON data
* Environment variable (JWKS_URL)

```python
# Using direct JWKS JSON
jwt_validator = LangchainJWTValidationTool(
    jwks_json={
        "keys": [
            {
                "kid": "key-id",
                "kty": "RSA",
                ...
            }
        ]
    }
)
```
## Permissions Check Tool
The Permissions Check tool integrates with Permit.io to verify user permissions against resources.

### Basic Usage

```python
from permit import Permit
from langchain_permit.tools import LangchainPermissionsCheckTool

# Initialize Permit client
permit_client = Permit(
    token="your_permit_api_key",
    pdp="http://localhost:7766"  # Your PDP URL
)

# Initialize the tool
permissions_checker = LangchainPermissionsCheckTool(
    permit=permit_client
)

# Check permissions
async def check_user_permission():
    result = await permissions_checker._arun(
        user={
            "key": "user-123",
            "firstName": "John"
        },
        action="read",
        resource={
            "type": "Document",
            "tenant": "default"
        }
    )
    print("Permission granted:", result)
```

### Input Formats
The permissions checker accepts different input formats:

1. Simple string for user (converts to user key):

```python
result = await permissions_checker._arun(
    user="user-123",
    action="read",
    resource="Document"
)
```

2. Full user object:

```python
result = await permissions_checker._arun(
    user={
        "key": "user-123",
        "firstName": "John",
        "lastName": "Doe",
        "email": "john@example.com",
        "attributes": {"department": "IT"}
    },
    action="read",
    resource={
        "type": "Document",
        "key": "doc-123",
        "tenant": "techcorp",
        "attributes": {"confidentiality": "high"}
    }
)
```

## Complete Example
Here's a complete example combining both tools:

```python
from langchain_permit.tools import LangchainJWTValidationTool, LangchainPermissionsCheckTool
from permit import Permit

# Initialize tools
jwt_validator = LangchainJWTValidationTool(
    jwks_url="http://localhost:3458/.well-known/jwks.json"
)

permit_client = Permit(
    token="permit_key_xxxxx",
    pdp="http://localhost:7766"
)

permissions_checker = LangchainPermissionsCheckTool(
    permit=permit_client
)

async def secure_document_access(jwt_token: str):
    # First validate the JWT
    try:
        claims = await jwt_validator._arun(jwt_token)
        print("Token validated successfully!")
        
        # Then check permissions using the claims
        user = {
            "key": claims['sub'],
            "firstName": claims['first_name']
        }
        
        resource = {
            "type": "Document",
            "tenant": claims.get('tenant', 'default')
        }
        
        allowed = await permissions_checker._arun(
            user=user,
            action="read",
            resource=resource
        )
        
        return {
            "access_granted": allowed,
            "user": claims['sub'],
            "tenant": claims.get('tenant')
        }
        
    except Exception as e:
        print(f"Access check failed: {str(e)}")
        return None
```

This documentation demonstrates the key features and usage patterns of both tools.

## Additional Demo Scripts

For fully runnable demos, check out the `examples/` folder in this repository. You’ll find:

* `demo_jwt_validation.py` – A quick script showing how to validate JWTs using `LangchainJWTValidationTool`.

* `demo_permissions_check.py` – A script that performs Permit.io permission checks using `LangchainPermissionsCheckTool`.

Just run `python demo_jwt_validation.py` or `python demo_permissions_check.py` (after setting your environment variables) to see these tools in action.